## Step1 导包

In [3]:
import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification

## Step2 加载数据集

In [4]:
pwd

'e:\\studyLLM\\project01'

In [5]:
# 不能联网
# ner_datasets = load_dataset("peoples_daily_ner") 此代码不能使用
from datasets import DatasetDict
ner_datasets = DatasetDict.load_from_disk("../data/ner_data")
ner_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 4637
    })
})

In [6]:
ner_datasets["train"]

Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 20865
})

In [7]:
ner_datasets["train"][0]

{'id': '0',
 'tokens': ['海',
  '钓',
  '比',
  '赛',
  '地',
  '点',
  '在',
  '厦',
  '门',
  '与',
  '金',
  '门',
  '之',
  '间',
  '的',
  '海',
  '域',
  '。'],
 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0]}

In [8]:
ner_datasets["train"].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}

根据ner_datasets可以获取其中的ClassLabels

In [9]:
label_list = ner_datasets["train"].features["ner_tags"].feature.names # 根据ner_datasets["train"].features去获取序列的标签
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

## Step3 数据预处理

数据预处理可以分为： \
- 加载分词器 
- 标签映射

In [10]:
pwd

'e:\\studyLLM\\project01'

In [11]:
# 加载分词器：tokenizer
tokenizer = AutoTokenizer.from_pretrained("../hfl/chinese-macbert-base")
tokenizer

BertTokenizerFast(name_or_path='../hfl/chinese-macbert-base', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

根据前面的输出可以知道ner_datasets已经做了tokenize，而对于已经做好tokenize的数据要指定is_split_into_words=True

In [12]:
tokenizer(ner_datasets["train"][0]["tokens"], truncation=True,is_split_into_words=True) 

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [101, 3862, 7157, 3683, 6612, 1765, 4157, 1762, 1336, 7305, 680, 7032, 7305, 722, 7313, 4638, 3862, 1818, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

对于英文词来说是存在子词的，如下面的例子所示, interesting word明明只有两个单词，但是除了101，102之外，还分出了5个token

In [13]:
res = tokenizer("interesting word")
res 

{'input_ids': [101, 10673, 12865, 12921, 8181, 8681, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

因此为了解决这个问题，我们需要借助word_ids以实现标签映射

In [14]:
def process_funciton(examples):
    tokenized_examples = tokenizer(examples["tokens"], max_length=128, truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_examples.word_ids(batch_index=i)
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100) # 在交叉熵损失中会自动忽略-100值以忽略begin和end的id
            else:
                label_ids.append(label[word_id])
        labels.append(label_ids)
    tokenized_examples["labels"] = labels
    return tokenized_examples


In [15]:
tokenized_datasets = ner_datasets.map(process_funciton, batched=True)
tokenized_datasets["train"][0]

Map:   0%|          | 0/20865 [00:00<?, ? examples/s]

Map:   0%|          | 0/2319 [00:00<?, ? examples/s]

Map:   0%|          | 0/4637 [00:00<?, ? examples/s]

{'id': '0',
 'tokens': ['海',
  '钓',
  '比',
  '赛',
  '地',
  '点',
  '在',
  '厦',
  '门',
  '与',
  '金',
  '门',
  '之',
  '间',
  '的',
  '海',
  '域',
  '。'],
 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0],
 'input_ids': [101,
  3862,
  7157,
  3683,
  6612,
  1765,
  4157,
  1762,
  1336,
  7305,
  680,
  7032,
  7305,
  722,
  7313,
  4638,
  3862,
  1818,
  511,
  102],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0, -100]}

## Step4 创建模型

IOB2的分类是多分类任务，在创建模型时要注意指定num_labels

In [16]:
model = AutoModelForTokenClassification.from_pretrained("../hfl/chinese-macbert-base", num_labels=len(label_list))

d:\Programs\miniconda3\envs\llm\lib\site-packages\transformers\modeling_utils.py:442: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_lo

In [17]:
model.config.num_labels

7

## Step5 创建评估函数

In [18]:
seqeval = evaluate.load("seqeval_metric.py")
seqeval

EvaluationModule(name: "seqeval", module_type: "metric", features: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')}, usage: """
Produces labelling scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    suffix: True if the IOB prefix is after type, False otherwise. default: False
    scheme: Specify target tagging scheme. Should be one of ["IOB1", "IOB2", "IOE1", "IOE2", "IOBES", "BILOU"].
        default: None
    mode: Whether to count correct entity labels with incorrect I/B tags as true positives or not.
        If you want to only count exact matches, pass mode="strict". default: None.
    sample_weight: Array-like of sha

In [19]:
import numpy as np

def eval_metric(pred):
    predicitons, labels = pred
    predicitons = np.argmax(predicitons, axis=-1)

    # 将id转换为原始的字符串类型的标签
    true_predictions = [
        [label_list[p] for p, l in zip(prediciton, label) if l != -100]
        for prediciton, label in zip(predicitons, labels)
    ]
    true_labels = [
        [label_list[l] for p, l in zip(prediciton, label) if l != -100]
        for prediciton, label in zip(predicitons, labels)
    ]

    result = seqeval.compute(predictions=true_predictions, references=true_labels, mode="strict", scheme="IOB2")
    return {
        "f1": result["overall_f1"]
    }

In [20]:
import accelerate
print(accelerate.__version__)

0.26.0


## Step6 配置训练参数

In [22]:
args = TrainingArguments(
    output_dir="out_for_models",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    logging_steps=50,
    num_train_epochs=3
)

## Step7 创建训练

In [23]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=eval_metric,
    data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer)
)

## Step8 模型训练

In [24]:
trainer.train()

d:\Programs\miniconda3\envs\llm\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/981 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.2237, 'learning_rate': 4.745158002038736e-05, 'epoch': 0.15}
{'loss': 0.0451, 'learning_rate': 4.490316004077472e-05, 'epoch': 0.31}
{'loss': 0.039, 'learning_rate': 4.235474006116208e-05, 'epoch': 0.46}
{'loss': 0.0336, 'learning_rate': 3.980632008154944e-05, 'epoch': 0.61}
{'loss': 0.0288, 'learning_rate': 3.72579001019368e-05, 'epoch': 0.76}
{'loss': 0.0274, 'learning_rate': 3.4709480122324164e-05, 'epoch': 0.92}


  0%|          | 0/19 [00:00<?, ?it/s]

...

  0%|          | 0/19 [00:00<?, ?it/s]

{'eval_loss': 0.017176713794469833, 'eval_f1': 0.9575342465753426, 'eval_runtime': 34.7389, 'eval_samples_per_second': 66.755, 'eval_steps_per_second': 0.547, 'epoch': 3.0}


d:\Programs\miniconda3\envs\llm\lib\site-packages\transformers\trainer.py:2220: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(best_model_path, map_lo

{'train_runtime': 2227.7685, 'train_samples_per_second': 28.098, 'train_steps_per_second': 0.44, 'train_loss': 0.02849084740385003, 'epoch': 3.0}


TrainOutput(global_step=981, training_loss=0.02849084740385003, metrics={'train_runtime': 2227.7685, 'train_samples_per_second': 28.098, 'train_steps_per_second': 0.44, 'train_loss': 0.02849084740385003, 'epoch': 3.0})

## Step9 使用pipeline预测

In [25]:
from transformers import pipeline

# 使用pipeline进行推理时，指定id2label
model.config.id2label = {idx:label for idx, label in enumerate(label_list)}
model.config

BertConfig {
  "_name_or_path": "../hfl/chinese-macbert-base",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",

如果模型是基于GPU训练的，那么推理时要指定device,对于NER任务，可以指定aggregation_strategy为simple，得到具体的实体的结果，而不是token的结果

In [ ]:
# 不使用aggregation_strategy
ner_pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, device=0)
ner_pipe("小明在北京上班")

[{'entity': 'B-PER',
  'score': 0.65637666,
  'index': 1,
  'word': '小',
  'start': 0,
  'end': 1},
 {'entity': 'I-PER',
  'score': 0.7820203,
  'index': 2,
  'word': '明',
  'start': 1,
  'end': 2},
 {'entity': 'B-LOC',
  'score': 0.99921155,
  'index': 4,
  'word': '北',
  'start': 3,
  'end': 4},
 {'entity': 'I-LOC',
  'score': 0.99893695,
  'index': 5,
  'word': '京',
  'start': 4,
  'end': 5}]

In [44]:
# 使用aggregate_strategy
ner_pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, device=0, aggregation_strategy="simple")
# res = ner_pipe("小明在北京上班")
# res

In [45]:
# ner_result = {}
# x = "小明在北京上班"
# for r in res:
#     if r["entity_group"] not in ner_result:
#         ner_result[r["entity_group"]] = []
#     ner_result[r["entity_group"]].append(x[r["start"]: r["end"]])

# ner_result

根据上面的结果可以发现能识别出实体，但是存在问题，每个实体之间都存在空格，因此我们可以根据star和end取实际结果

In [ ]:
def get_real_result(x):
    ner_result = {}
    res = ner_pipe(x)
    for r in res:
        if r["entity_group"] not in ner_result:
            ner_result[r["entity_group"]] = []
        ner_result[r["entity_group"]].append(x[r["start"]: r["end"]])
    return ner_result

x = "小明在北京"
ner_result = get_real_result(x)
ner_result

{'PER': ['小明'], 'LOC': ['北京']}